# Hecho novedades

In [1]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml

### Conexión con la base de datos

In [2]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [3]:
dimension_fecha = pd.read_sql_table('dim_fecha', olap_conn)
dimension_mensajero = pd.read_sql_table('dim_mensajero', olap_conn)
tabla_novedadesservicio = pd.read_sql_table('mensajeria_novedadesservicio', oltp_conn)
tabla_tipo_novedades = pd.read_sql_table('mensajeria_tiponovedad', oltp_conn)
tabla_tipo_novedades

,id,nombre
0,2,No puedo continuar
1,1,Novedades del servicio


### Módulo de transformación

In [4]:
tabla_tipo_novedades.sort_values(by=['id'], inplace=True)
tabla_novedadesservicio['fecha'] = pd.to_datetime(tabla_novedadesservicio['fecha_novedad'].dt.date)
tabla_novedadesservicio['hora'] = pd.to_datetime(tabla_novedadesservicio['fecha_novedad'].dt.time.astype(str).str.split('.').str[0], format='%H:%M:%S').dt.time
#El desface de las horas es por la zona horaria, por eso no concuerdan directamente con tabla_novedadesservicio

hecho_novedades = (pd.merge(
  tabla_novedadesservicio[['fecha', 'hora', 'tipo_novedad_id']], 
  dimension_fecha[['fecha', 'hora', 'id']], # Podría agregar el cliente con el servicio_id
  how='left', 
  on=['fecha', 'hora'])
  .drop(columns=['hora'])
  .rename(columns={'id': 'id_fecha'}))

conteo_novedades = {}

for tipo_id in tabla_tipo_novedades['id']: # genera todas las llaves con la dimensión fecha 
  conteo_novedades[f'num_novedad_{tipo_id}'] = (hecho_novedades[hecho_novedades['tipo_novedad_id'] == tipo_id]
    .groupby('fecha')['tipo_novedad_id']
    .count())

# conteo total por tipo novedad
conteo_novedades = pd.DataFrame(conteo_novedades).fillna(0).astype(int)
# num_novedades como la suma de los conteos por tipo
conteo_novedades['total_novedades'] = conteo_novedades.sum(axis=1)

hecho_novedades = (hecho_novedades.merge(conteo_novedades, on='fecha', how='left')
  .drop_duplicates(subset=['fecha'])
  .drop(columns=['fecha', 'tipo_novedad_id'])
  .reset_index(drop=True))

hecho_novedades
#hecho_novedades.shape[0]

,id_fecha,num_novedad_1,num_novedad_2,total_novedades
0,128402,6,3,9
1,128411,2,1,3
2,128414,0,1,1
3,128415,1,3,4
4,128419,2,1,3
...,...,...,...,...
198,133370,62,8,70
199,133440,19,6,25
200,133465,70,10,80
201,133545,41,9,50


### Módulo de carga a la bodega de datos

In [5]:
try: # Realizar una consulta sencilla para verificar la conexión
    # Ejecutar una consulta para obtener el nombre de las tablas
    tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", olap_conn)
    #print(tables)
    print("Conexión establecida y consulta realizada con éxito.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión establecida y consulta realizada con éxito.


In [6]:
try:
  hecho_novedades.to_sql('hecho_novedades', olap_conn, if_exists='replace', index_label='id')
except Exception as e:
  print(f"Error al cargar datos: {e}")